In [2]:
#for trans_image_model
import numpy as np
from skimage import transform

# for face_align
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
import imutils
import dlib
import cv2

# 모델들이 사용할 수 있게 읽어온 이미지를 원하는 array로 변환
def trans_image_model(image):
    # cv2는 이미지를 읽어올때 GBR로 읽어온다. 이를 RGB로 변환
    image_for_model = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_for_model = np.array(image_for_model).astype('float32')/255

    # 안경, 선글라스, 마스크 모델용 이미지
    # RGB 3채널 + 학습사진크기 : 128 * 128
    RGBimage = transform.resize(image_for_model, (128, 128, 3))
    RGBimage = np.expand_dims(RGBimage, axis=0)

    # 감정 모델용 이미지 
    # gray 스케일로 인한 1채널 + 학습사진크기 : 48 * 48 
    GRAYimage = transform.resize(image_for_model, (48, 48, 1))
    GRAYimage = np.expand_dims(GRAYimage, axis=0)
   
    return (RGBimage, GRAYimage)


def face_align(image):
    predictor_model = "./landmarks.dat"
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_model)
    fa = FaceAligner(predictor, desiredFaceWidth=256)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 2)

    for rect in rects:
        (x, y, w, h) = rect_to_bb(rect)
        faceOrig = imutils.resize(image[y:y + h, x:x+w], width=256)
        faceAligned = fa.align(image, gray, rect)
        return faceAligned

# 이미지에서 얼굴인식하고 추출
def reg_face(image):
    
    prototxtPath = "./face_detector/deploy.prototxt"
    weightsPath = "./face_detector/res10_300x300_ssd_iter_140000.caffemodel"
    faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

    (h, w) = image.shape[:2]
    
    blob = cv2.dnn.blobFromImage(image, 1.0, (224, 224),
		(104.0, 177.0, 123.0))
    faceNet.setInput(blob)
    detections = faceNet.forward()

	# initialize our list of faces, their corresponding locations,
	# and the list of predictions from our face mask network
    locs = []

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the detection
        confidence = detections[0, 0, i, 2]

	    # filter out weak detections by ensuring the confidence is
	    # greater than the minimum confidence
        if confidence > 0.5:
	        # compute the (x, y)-coordinates of the bounding box for
	        # the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
	    	# ensure the bounding boxes fall within the dimensions of
	    	# the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# extract the face ROI, convert it from BGR to RGB channel
			# ordering, resize it to 224x224, and preprocess it
            face = image[startY:endY, startX:endX]
            locs.append((startX, startY, endX, endY))
    
    # only make a predictions if at least one face was detected 
    if len(locs) > 1:
        return -1

    return image[locs[0][1]:locs[0][3], locs[0][0]:locs[0][2]]

In [19]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

def emotion():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.load_weights("./model/emotion_model.h5")
    return model

def mask():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation = 'relu', input_shape = (128, 128, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(64,kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(128,kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation = 'sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='Adamax', metrics=['accuracy'])
    model.load_weights("./model/mask_model_ver3.h5")
    return model

def glasses():
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3), activation = 'relu', input_shape = (128, 128, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(64,kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(128,kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation = 'sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='Adamax', metrics=['accuracy'])
    model.load_weights("./model/mask_glasses_model_final3.h5")
    return model

def sunglasses():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation = 'relu', input_shape = (128, 128, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(64,kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(128,kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation = 'sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='Adamax', metrics=['accuracy'])
    model.load_weights("./model/mask_sunglasses_model_final.h5")
    return model


In [20]:
def skin():
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3), activation = 'relu', input_shape = (128, 128, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(64,kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(128,kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation = 'softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='Adamax', metrics=['accuracy'])
    model.load_weights("./model/skin_model_batch_50_epoch_100.h5")

    return model

In [21]:
def roundrec():

    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3), activation = 'relu', input_shape = (128, 128, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(64,kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(128,kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation = 'sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='Adamax', metrics=['accuracy'])
    model.load_weights("./model/mask_round_rectangle_model_final.h5")
    
    return model

In [24]:
import numpy as np
import tensorflow as tf
import cv2
import math

# 학습된 모델 가져오기
model_skin = skin()
model_glasses = glasses()
#model_sunglasses = sunglasses()
#model_round = roundrec()
model_mask = mask()


# 대상 사진파일경로
#file_path = '../graduation_dataset/race_test/asian_028.png'
#file_path = '../graduation_dataset/race_test/black_023.png'
#file_path = '../graduation_dataset/race_test/white_004.png'
#file_path = '../graduation_dataset/glasses/test_3/rectangle_002.png'
#file_path = '../graduation_dataset/test/black (8).jpg'
#file_path = '../graduation_dataset/mask/train4/images/4118.png'
#file_path = '../graduation_dataset/mask/train3/nomask_8020.jpg'
file_path = '../grauation/face/aaa (2).jpg'
# 대상사진
image = cv2.imread(file_path)

image2 = reg_face(image)

RGBimage, GRAYimage = trans_image_model(image2)
RGBimage2, GRAYimage2 = trans_image_model(GRAYimage)



# cv2.imshow("..", RGBimage[0])
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# #RGBimage[0].show()
# print(RGBimage[0])

result = model_glasses.predict(RGBimage)

# print('예측: ')
# if max(result[0]) == result[0][0]:
#     print('asian')
# elif max(result[0]) == result[0][1]:
#     print('black')
# else:
#     print('white')
        
# result[0][0] = int(round(result[0][0],3)*10000)
# result[0][1] = int(round(result[0][1],3)*10000)
# result[0][2] = int(round(result[0][2],3)*10000)
# result[0][3] = int(round(result[0][3],3)*10000)
# result[0][4] = int(round(result[0][4],3)*10000)
# result[0][5] = int(round(result[0][5],3)*10000)
            
print(result[0])

# result_glasses = model_glasses.predict(RGBimage)
# if max(result_glasses[0]) == result_glasses[0][0]:
#     print('glasses')
# else:
#     print('noglasses')
# print(result_glasses)

# # 피부색      -> 피부색에 따라                     
# # 마스크      -> no 입     -> mask              => 0 : 입 , 1: 마스크
# #           -> no 감정표현 -> 눈 : netural      => default 를 netural로
# # 안경       -> 안경추가                        => 0: 안경X , 1: 안경O


[0.9563279  0.04543835]
